Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [122]:
NAME = "Ian McGregor"

---

<a id='top'></a>

# CSCI 3202, Spring 2021:  Assignment 5  

Shortcuts:  [top](#top) -- [1](#p1) | [1a](#p1a) | [1b](#p1b) | [1c](#p1c) | [1d](#p1d) | [1e](#p1e) | [1f](#p1f) | [1g](#p1g) -- [2](#p2) | [2a](#p2a) | [2b](#p2b) | [2c](#p2c) | [2d](#p2d) | [2e](#p2e) 

# Assignment Overview

This assignment is an exercise in implementing and analyzing Markov Decision Processes (MDPs). Problem 1 asks you to code a solution to a specific scenario, while Problem 2 is a conceptual question which asks you to describe an MDP problem of your own design.

Here's a summary of the tasks required and the associated points:

| Problem #  | Tasks                                                  | Points  |
|:---        |:---                                                    |:---:    |
| 1a         | Code: Complete implementation of `MDP` class           | 10      |
| 1b         | Code: Implement `value_iteration` and `find_policy`    | 5       |
| 1c         | Code and create: Generate and illustrate optimal path  | 5       |
| 1d         | Written: analyze policy                                | 5       |
| 1e         | Code: adjust non-terminal rewards                      | 5       |
| 1f         | Code and write: adjust terminal rewards                | 5       |
| 1g         | Written: analyze transition model                      | 5       |
| 2a         | Written: define problem                                | 4       |
| 2b         | Written: define states                                 | 4       |
| 2c         | Written: define reward                                 | 4       |
| 2d         | Written: define actions and transition                 | 4       |
| 2e         | Written: define optimal policy                         | 4       |
| Total      |                                                        | 60      |


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
# add any imports you may need

<a id='p1'></a>
[Back to top](#top)

# Problem 1: Navigating an awkward situation with grace and poise

<img src='https://www.explainxkcd.com/wiki/images/5/5f/interaction.png' style="width: 600px;"/>

Suppose you are at a social event where you would like to avoid any interaction with a large number of the other attendees. It's not that you don't like them, it's just that you don't like *talking to* them. A few of your good friends are also in attendance, but they are tucked away in a corner. The rectangular room in which the event is being held spans gridcells at $x=1,2,\ldots, 6$ and $y=1,2,\ldots, 5$. At the eastern edge ($x=6$) of this first floor room, there is a balcony, with a 6-foot drop. If the event becomes unbearably awkward, you can jump off the balcony and run away. Of course, this might hurt a little bit, so we should incorporate this into our reward structure.

The terminal states and rewards associated with them are given in the diagram below. The states are represented as $(x,y)$ tuples. The available actions in non-terminal states include moving exactly 1 unit North (+y), South (-y), East (+x) or West (-x), although you should not include walking into walls, because that would be embarrassing in front of all these other people. Represent actions as one of 'N', 'S', 'E', or 'W'. For now, assume all non-terminal states have a default reward of -0.01, and use a discount factor of 0.99.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/hw06_mdp.png" style="width: 400px;"/>

Use the following transition model for this decision process, if you are trying to move from state $s$ to state $s'$:
* you successfully move from $s$ to $s'$ with probability 0.6
* the remaining 0.4 probability is spread equally likely across state $s$ **and** all adjacent (N/S/E/W) states except for $s'$. Note that this does not necessarily mean that all adjacent states have 0.1, because some states do not have 4 adjacent states.

<a id='p1a'></a>
[Back to top](#top)

## (1a) - 10 points

Complete the `MDP` class below. The docstring comments provide some desired specifications. You may add additional methods or attributes, if you would like.

In [2]:
class MDP:
    def __init__(self, nrow, ncol, terminal_states, default_reward, df):
        
        # self.states -- a list of all the states as (x,y) tuples
        self.states = list()
        if not self.states:
            for i in range(ncol):
                for j in range(nrow):
                    self.states.append((i+1,j+1))
        # self.terminal_states -- a dictionary with terminal state keys, and rewards as values
        self.terminal_states = terminal_states
        # self.default_reward -- the reward for being in any non-terminal state
        self.default_reward = default_reward
        # self.df -- discount factor
        self.df = df
        self.maxrow = nrow
        self.maxcol = ncol
        self.path = list()

    def actions(self, state):
        '''Return a list of available actions from the given state.
        Possible actions are 'N','S','E','W'
        [None] are the actions available from a terminal state.
        '''
        if(state in self.terminal_states):
            return [None]
        else:
            x, y = state
            actions = ['N', 'S', 'E', 'W']
            if(x == 1):
                actions.remove('W')
            if(y == 1):
                actions.remove('S')
            if(x == self.maxcol):
                actions.remove('E')
            if(y == self.maxrow):
                actions.remove('N')
            return actions
        
    def reward(self, state):
        '''Return the reward for being in the given state'''
        if(state in self.terminal_states):
            return self.terminal_states[state]
        else:
            return self.default_reward
        
    def result(self, state, action):
        '''Return the resulting state (as a tuple) from doing the given
        action in the given state, without uncertainty. Uncertainty
        is incorporated into the transition method.
        state -- a tuple representing the current state
        action -- one of N, S, E or W, as a string
        '''
        x, y = state
        if(action == 'Stay'):
            return(x,y)
        assert action in self.actions(state), 'Error: action needs to be available in that state'
        assert state in self.states, 'Error: invalid state'
        
        if(action == 'N'):
            y = y + 1
            return (x,y)
        elif(action == 'S'):
            y = y - 1
            return (x,y)
        elif(action == 'E'):
            x = x + 1
            return (x,y)
        elif(action == 'W'):
            x = x - 1
            return (x,y)


    def transition(self, state, action):
        '''Return a list of (probability, next_state) associated
        with the possibilities of taking the given action from the given state.
        '''
        if action is None:
            # This happens for a terminal state
            return [(0, state)]
        else:
            # Make a list of actions available
            actionList = self.actions(state)
            actionList.append('Stay')
            # Make an empty list to contain future states
            futureStates = []
            for i in range(len(actionList)):
                # If action in list is equal to desired action, add resulting state with probability 0.6 to future state list
                if (actionList[i] == action):
                    futureStates.append((0.6, self.result(state, actionList[i])))
                # Not the desired action
                else:
                    futureStates.append((0.4/(len(actionList) -1), self.result(state, actionList[i])))
            return futureStates

    def expected_utility(self, next_states, cur_util):
        '''Return the expected utility given generated list of possible 
        next states and the current utility, which is a dictionary of the form {state : utility}
        '''
        sum = 0
        
        for i in range(len(next_states)):
            probability, state = next_states[i]
            sum = sum + (probability * cur_util[state])
        return sum

### (1a) tests

Note that these are non-exhaustive, because there is some flexibility in how the `transition` method works.

In [3]:
## BEGIN TESTS
nrow = 3
ncol = 3
default_reward = -0.2
discount = 0.5
terminal = {(1,3):-1, (1,2):2}
mdp_simple = MDP(nrow, ncol, terminal, default_reward, discount)

actions1 = set(mdp_simple.actions((2,2)))
assert (actions1 == {'N','S','E','W'}), "Expected set of actions is {'N','S','E','W'}, your code returned: %s" % actions1

actions2 = set(mdp_simple.actions((1,1)))
assert (actions2 == {'N','E'}), "Expected set of actions is {'N','E'}, your code returned: %s" % actions2

actions3 = set(mdp_simple.actions((1,2)))
assert (actions3 == {None}), "Expected set of actions is {None}, your code returned: %s" % actions3

reward1 = mdp_simple.reward((1,2))
assert (reward1 == 2), "Expected reward is 2, your code returned: %f" % reward1

reward2 = mdp_simple.reward((2,2))
assert (reward2 == -0.2), "Expected reward is -0.2, your code returned: %f" % reward2

result1 = mdp_simple.result((1,1), 'N')
assert (result1 == (1,2)), "Expected result is (1,2), your code returned: %s" % (result1,)

print("All tests passed: 5 points")
## END TESTS

All tests passed: 5 points


In [4]:
## BEGIN TESTS
# set values from problem statement
nrow = 5
ncol = 6
default_reward = -0.01
discount = 0.99
terminal = {(1,3):-1, (1,4):2, (1,5):2, (2,1):-1, (3,1):-1, (3,4):-1, (3,5):1,
            (4,3):-1, (4,4):-1, (6,1):-5, (6,2):-5, (6,3):-5, (6,4):-5, (6,5):-5}
mdp_p1 = MDP(nrow, ncol, terminal, default_reward, discount)

# Find the expected utility of walking N from (1,1):
util_old = {s : s[0]+s[1] for s in mdp_p1.states}

next_states = mdp_p1.transition((1,1), 'N')
assert (len(next_states) == 3), "Expected 3 possible next states, your code returned: %d" % len(next_states)

exp_util = mdp_p1.expected_utility(next_states, util_old)
assert (exp_util == 2.8), "Expected utility should be 2.8, your code returned %f" % exp_util
print("All tests passed: 5 points")
## END TESTS

All tests passed: 5 points


<a id='p1b'></a>
[Back to top](#top)

## (1b) - 5 points

Implement value iteration to calculate the utilities for each state.  Also implement a function that takes as arguments an `MDP` object and a dictionary of state-utility pairs (key-value) and returns a dictionary for the optimal policy.  The optimal policy dictionary should have state tuples as keys and the optimal move (None, N, S, E or W) as values.

In [5]:
def value_iteration(mdp, tol=1e-3):
    u = {}
    uPrime = {}
    gamma = 0.99
    # 1. initialize utility for all states to 0 -> U
    for n in range(len(mdp.states)):
        uPrime[mdp.states[n]] = 0
        u[mdp.states[n]] = 0
    # Repeat 2 until problem is converged
    counter = 0
    while(True):
        delta = 0
        for a in range(len(u)):
            u[mdp.states[a]] = uPrime[mdp.states[a]]
        # 2. for each state on the board
        for i in range(len(mdp.states)):
            #    2.1. calculate expected utility for each possible next state
            # List of actions for current state
            actions = mdp.actions(mdp.states[i])
            # R(s)
            reward = mdp.reward(mdp.states[i])
            # List of utility scores
            futureUtil = []
            for j in range(len(actions)):
                next_states = mdp.transition(mdp.states[i], actions[j])
                util = mdp.expected_utility(next_states, u)
                futureUtil.append(util)
                #    2.2. find best utility out of possible expected utilities
                #    2.3. define new utility of the state
            uPrime[mdp.states[i]] = (max(futureUtil)*gamma) + reward
            # update delta
            if((abs(uPrime[mdp.states[i]] - u[mdp.states[i]]) > delta)):
                delta = abs(uPrime[mdp.states[i]] - u[mdp.states[i]])
        if(delta < tol * ((1-gamma)/gamma)):
            return u

def find_policy(mdp, utility):
    '''Return a dictionary containing the best policy for each state s,
    of the form {s : s_policy}
    '''
    pi = {}
    # 1. initialize utility for all states to '' -> U
    for n in range(len(mdp.states)):
        pi[mdp.states[n]] = ''
    # For each state on the board
    for i in range(len(mdp.states)):
        #    Calculate optimal policy at each state
        #    List of actions for current state
        actions = mdp.actions(mdp.states[i])
        curUtil = -5
        bestAction = ''
        for k in range(len(actions)):
            if(actions[k] != None and curUtil < utility[mdp.result(mdp.states[i], actions[k])]):
                curUtil = utility[mdp.result(mdp.states[i], actions[k])]
                # curUtil = sum([p*utility[s2] for p, s2 in mdp.transition(mdp.states[i],actions[k])])
                bestAction = actions[k]
        pi[mdp.states[i]] = bestAction
    return pi

### (1b) tests

In [6]:
## BEGIN TESTS

utility = value_iteration(mdp_p1, tol=1e-6)
policy = find_policy(mdp_p1, utility)

util1 = utility[(1,5)]
assert (util1 == 2), "Expected utility of 2, your code returned %f" % util1

util2 = utility[(6,1)]
assert (util2 == -5), "Expected utility of -5, your code returned %f" % util2

util3 = round(utility[(2,5)],2)
assert (util3 == 1.74), "Expected utility of 1.74, your code returned %f" % util3

util4 = round(utility[(5,3)],2)
assert (util4 == -1.39), "Expected utility of -1.39, your code returned %f" % util4

policy1 = policy[(2,4)]
assert (policy1 == 'W'), "Expected policy is 'W', your code returned %s" % policy1

policy2 = policy[(1,1)]
assert (policy2 == 'N'), "Expected policy is 'N', your code returned %s" % policy2

print("Passed test cases: 5 points")
## END TESTS

Passed test cases: 5 points


<a id='p1c'></a>
[Back to top](#top)

## (1c) - 5 points

If we enter the room at $s_0$, what is the optimal path for us to follow? Complete the following function to generate the sequence of states along the path. If we start in state $s_0$, then your output should be in the form $[s_0, s_1, s_2, ... , s_{term}]$ where $s_{term}$ is a terminal state. Set your tolerance for value iteration to be $10^{-6}$

Additionally, create a graphic to illustrate this policy pathway, either by generating a plot in Python or by uploading a hand-drawn image and including a link to it below.

In [7]:
def find_optimal_path(mdp, state):

    utility = value_iteration(mdp, tol=1e-6)
    policy = find_policy(mdp, utility)
    next_state = mdp.result(state, policy[state])
    path = list()
    path.append(state)
    while(True):
        path.append(next_state)
        next_state = mdp.result(next_state, policy[next_state])
        if(next_state in mdp.terminal_states):
            path.append(next_state)
            return path
    


Put a link to your graphic below for the path from (5,3). You can find an [example image here](http://www.cs.colorado.edu/~tonyewong/home/resources/hw06_mdp_path.png) with the optimal path starting from (5,1). **Please include a link rather than attaching a file**. This can be a link to your file in Google Drive, with the permissions set to public. The graders will not ask for permissions! The syntax for including a link in markdown is `[link text](url.com/to/your/image)`

[Masterpiece](https://drive.google.com/file/d/1kP5M4KjHIowt_7-Lgy2tiCS-rKO-uk6b/view?usp=sharing)

### (1c) tests

In [8]:
## BEGIN TESTS
path1 = find_optimal_path(mdp_p1, (5,3))
assert (path1 == [(5, 3), (5, 2), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)]), "The optimal path is [(5, 3), (5, 2), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)], your code generated %s" % path1

path2 = find_optimal_path(mdp_p1, (5,1))
assert (path2 == [(5, 1), (4, 1), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)]), "The optimal path is [(5, 1), (4, 1), (4, 2), (3, 2), (2, 2), (2, 3), (2, 4), (1, 4)], your code generated %s" % path2

path3 = find_optimal_path(mdp_p1, (1,1))
assert (path3 == [(1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (1, 4)]), "The optimal path is [(1, 1), (1, 2), (2, 2), (2, 3), (2, 4), (1, 4)], your code generated %s" % path3

print("Passed tests: 3 points")
## END TESTS

Passed tests: 3 points


<a id='p1d'></a>
[Back to top](#top)

## (1d) - 5 points

From (3,2) the optimal move is to walk West. If we are trying to go talk to our friends in the Northwest corner, why would we rather do this than walk North first, then West?

If we were to move north to (3,3) we would be neighboring 2 people whom we don't exactly want to talk to, which leaves room for error in movement. If there is an error in movement, then we may end up accidentally talking to them, which is precisely what we are trying to avoid. 

<a id='p1e'></a>
[Back to top](#top)

## (1e) - 5 points

How painfully awkward do you need to set the default reward for non-terminal states before the optimal move at (5,1) becomes jumping off the balcony immediately and running away? Implement the following function which returns the reward where the policy for (5,1) is to jump off the balcony.

In [9]:
def find_non_terminal_reward():
    nrow = 5
    ncol = 6
    discount = 0.99
    terminal = {(1,3):-1, (1,4):2, (1,5):2, (2,1):-1, (3,1):-1, (3,4):-1, (3,5):1,
                (4,3):-1, (4,4):-1, (6,1):-5, (6,2):-5, (6,3):-5, (6,4):-5, (6,5):-5}
    for reward in np.arange(-0.01, -3, -0.01):
        mdp = MDP(nrow, ncol, terminal, reward, discount)
        utility = value_iteration(mdp, tol=1e-3)
        policy = find_policy(mdp, utility)
        if (policy[5,1] == ''):
            return reward  

### (1e) tests

In [10]:
reward1 = find_non_terminal_reward()
assert (reward1 == -2.09), "The expected reward is -2.09, your code returned %f" % reward1

print("Test passed: 5 points")

Test passed: 5 points


<a id='p1f'></a>
[Back to top](#top)

## (1f) - 5 points

In **1e** we assumed a certain level of loss (negative reward) just for being present.  But a more realistic approach might be to instead change the reward structure for the terminal states. Consider the terminal states with -1 reward in the default model. Let $R^*$ denote the reward associated with these states. How low does $R^*$ need to be in order for us to immediately jump off the balcony and run away? Use the default non-terminal state reward of -0.01. Implement the following function to return the value of $R^*$ which leads to a policy of jumping off the balcony at (5,1). Write a few sentences interpreting your result.

In [11]:
def find_terminal_reward():
    nrow = 5
    ncol = 6
    discount = 0.99
    default_reward = -0.01
    for rstar in np.arange(-6, -12, -0.01):
        # 1. set the reward of the terminal nodes 
        terminal = {(1,3):rstar, (1,4):2, (1,5):2, (2,1):rstar, (3,1):rstar, (3,4):rstar, (3,5):1,
                (4,3):rstar, (4,4):rstar, (6,1):-5, (6,2):-5, (6,3):-5, (6,4):-5, (6,5):-5}
        # 2. define MDP with appropriate parameters
        mdp = MDP(nrow, ncol, terminal, default_reward, discount)
        utility = value_iteration(mdp, tol=1e-3)
        policy = find_policy(mdp, utility)
        # 3. find policy for state (5,1)
        if (policy[5,1] == ''):
            # 4. return R* if the policy for (5,1) is to jump off the balcony
            pprint.pprint(utility)
            return rstar

### (1f) tests

In [12]:
reward1 = round(find_terminal_reward(),2)
assert (reward1 == -11.39), "Expected reward is -11.39, your code returned %f" % reward1

print("Passed test: 3 points")

{(1, 1): -6.833459774489048,
 (1, 2): -5.41283352156081,
 (1, 3): -11.389999999999885,
 (1, 4): 2.0,
 (1, 5): 2.0,
 (2, 1): -11.389999999999885,
 (2, 2): -3.8431806854447985,
 (2, 3): -2.1921451600621142,
 (2, 4): -0.019436479414799,
 (2, 5): 1.5062608910461857,
 (3, 1): -11.389999999999885,
 (3, 2): -4.92015112919688,
 (3, 3): -4.49994054313204,
 (3, 4): -11.389999999999885,
 (3, 5): 1.0,
 (4, 1): -6.121672307468489,
 (4, 2): -5.7283760765497105,
 (4, 3): -11.389999999999885,
 (4, 4): -11.389999999999885,
 (4, 5): -1.4137848707730587,
 (5, 1): -5.124568310657509,
 (5, 2): -5.000516701460532,
 (5, 3): -4.925974296181743,
 (5, 4): -3.8899720740860646,
 (5, 5): -2.3309495896804235,
 (6, 1): -5.0,
 (6, 2): -5.0,
 (6, 3): -5.0,
 (6, 4): -5.0,
 (6, 5): -5.0}
Passed test: 3 points


Write a few sentences with your interpretation here:

I have output the utility using pretty print for reference. As you can see if we are starting at position (5,1) the utilities for the surrounding states are: <br>
    -- (5, 2): -5.000516701460532 <br>
    -- (4, 1): -6.121672307468489 <br>
    and finally <br>
    -- (6, 1): -5.0 <br>
    The optimal policy would be to throw yourself off the balcony as the rest of the room must be either: filled with bees or literally on fire. <br>
    No but seriously, as the pitfalls in the room become increasingly consequential, our value iteration, and policy iteration algorithms compute that the risk of falling into any of these pitfalls accidentally is far more dangerous than jumping off the balcony. Some examples that come to mind could be as I stated before, maybe a couple of killer hornet nests are in the room, or DEFINITELY COVID POSISTIVE people without masks are literally coughing incessantly in the room. Whatever the case may be, as we iterate over all possible states and actions the correct decision becomes effortless and clear, talking to your friends is not worth dying for. This concludes my TedTalk.

<a id='p1g'></a>
[Back to top](#top)

## (1g) - 5 points

Given the problem context, write a few sentences about why this is or is not an appropriate transition model. Include an interpretation of the terminal states.

I think the transition makes perfect sense. In the context of socializing at a party, I would find myself weighing these exact same decisions. I'm pretty particular about whom I talk to, and I am strongly averse to engaging in small talk. (Sure waste the only true non-renewable resource by talking about the weather - time) <br>
The terminal states that are not jumping off of the balcony are pretty accurate, while you're certainly not going to break your leg(-5), talking to boring people is pretty unpleasant (-1). I would say that acheiving the goal state, and egaging in real conversation is pretty pleasurable, and in the context of a large party, very enticing (+2). If we are measuring the rewards at all terminal states by their relativity to each other, I would say that talking to my friends is not a +5, that is it is not the opposite of breaking my leg (-5) but +2 would be a reasonable measurement, as well as a -1 for egaging in boring conversation. 

---
<a id='p2'></a>
[Back to top](#top)

# Problem 2: Define your very own MDP

For this problem, you do not need to write any code, but rather communicate your ideas clearly using complete sentences and descriptions of the concepts the questions ask about. You can, of course, include some pseudocode if it helps, but that is not strictly necessary.


<a id='p2a'></a>

## (2a) - 4 points

Describe something you think would be interesting to model using a Markov decision process.  Be **creative** - do not use any examples from your homework, class, or the textbook, and if you are working with other students, please **come up with your own example**. There are so, SO many possible answers!

I was thinking of this question while I was grocery shopping last night and I realized, grocery shopping is absolutely perfect for an MDP model. Your living reward would be negative, and meausured in the amount of time you spend at each state. This example is of course limited to grocery shopping in a very specific context, lets say you have 5 minutes to find ben and jerrys ice cream for your very intoxicated and hangry friend, and they will only settle for Half Baked because they can't decide if they want chocolate vs vanilla ice cream, or cookie dough vs brownie batter. Naturally of course, Half Baked is the go-to, coming in second only to Boots on the Moon, which is my favorite flavor, but that is beside the point! ... Or is it?

<a id='p2b'></a>

## (2b) - 4 points

What are the states associated with your MDP? Include a discussion of terminal/non-terminal states.

The states include the start state (entrance to the grocery store), the goal state (the freezer location that contains Half Baked), and the various pitfalls (Boots on the Moon of course, because your time is so limited, you don't have time to shop for ourselves), the chip aisle(again very distracting), and finally wherever they keep the bacon (duh).

<a id='p2c'></a>

## (2c) - 4 points

What is the reward structure associated with your MDP?  Include a discussion of terminal/non-terminal states.

The distracted states would end up in a terminal state with reward -5 and the location of Half Baked would have a whopping +10 reward. Of course the living reward is -.5 as time is precious and grocery stores are massive time sinks, and just plain massive. Discount of future rewards would 0.99.

<a id='p2d'></a>

## (2d) - 4 points

What are the actions and transition model associated with your MDP?

The actions are move N,S,E,W and the transition model would depend on the various obstacles in your path such as other shoppers, aisles and various coolers placed directly in your path of acheiving the Half Baked extraction as fast as possible.

<a id='p2e'></a>

## (2e) - 4 points

Interpret what an optimal policy represents in the context of your particular MDP.

The optimal policy would be to stay as far away from the distraction states, one of which is very close to your goal state, so we must ensure we enter the ice cream aisle from the opposite end of Boots on the Moon, as well as avoiding any obstacles. 